## Section 1: Import Required Libraries and Setup

In [ ]:
import json
import pandas as pd
import numpy as np
import sqlite3
import duckdb
import os
import time
from datetime import datetime
from typing import Dict, List, Tuple, Any
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

# Configure paths
PROJECT_DIR = r'c:\Users\mvzie\Documents\AI Agent Experiment'
DB_PATH = os.path.join(PROJECT_DIR, 'animal_shelter.duckdb')
GROUND_TRUTH_PATH = os.path.join(PROJECT_DIR, 'agent_ground_truth_test_cases.json')

print(f"Project Directory: {PROJECT_DIR}")
print(f"Database Path: {DB_PATH}")
print(f"Ground Truth Path: {GROUND_TRUTH_PATH}")
print(f"\nFiles exist:")
print(f"  Database: {os.path.exists(DB_PATH)}")
print(f"  Ground Truth: {os.path.exists(GROUND_TRUTH_PATH)}")

## Section 2: Load Ground Truth Test Cases

In [ ]:
# Load ground truth test cases
with open(GROUND_TRUTH_PATH, 'r') as f:
    test_cases_data = json.load(f)

print(f"Project: {test_cases_data['project']}")
print(f"Description: {test_cases_data['description']}")
print(f"Total Test Cases: {test_cases_data['total_test_cases']}")
print(f"\nTest Cases Loaded:")
for tc in test_cases_data['test_cases']:
    print(f"  Q{tc['id']}: {tc['name']} ({tc['result_count']} expected rows)")

In [ ]:
# Organize test cases for easier access
test_cases = {tc['id']: tc for tc in test_cases_data['test_cases']}
test_ids = sorted(test_cases.keys())

print(f"\nTest Case Q11 (Key Metric):")
q11 = test_cases[11]
print(f"  Question: {q11['natural_language_question']}")
print(f"  Expected Rows: {q11['result_count']}")
print(f"  Expected Results:")
for row in q11['expected_results']:
    print(f"    {row['age_group']}: {row['spayed_neutered_pct']}% spayed/neutered")

## Section 3: Connect to DuckDB and Verify Schema

In [ ]:
# Connect to DuckDB
conn = duckdb.connect(DB_PATH)

# Verify tables exist
tables = conn.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='main'").fetchall()
table_names = [t[0] for t in tables]

print(f"Tables in DuckDB ({len(table_names)}):")
for table in sorted(table_names):
    count = conn.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"  {table}: {count:,} rows")

# Verify key tables for validation
required_tables = ['fact_animal_outcome', 'dim_outcome_type', 'dim_date', 
                    'dim_animal_attributes', 'dim_sex_on_outcome', 'dim_intake_details']
print(f"\nRequired tables present: {all(t in table_names for t in required_tables)}")

## Section 4: Execute Ground Truth SQL Queries and Capture Results

In [ ]:
# Execute all ground truth queries and store results
ground_truth_results = {}
execution_times = {}

print("Executing ground truth SQL queries...\n")

for test_id in test_ids:
    test = test_cases[test_id]
    start_time = time.time()
    
    try:
        result_df = conn.execute(test['ground_truth_sql']).df()
        execution_times[test_id] = time.time() - start_time
        ground_truth_results[test_id] = result_df
        
        print(f"Q{test_id}: {test['name']}")
        print(f"  Rows: {len(result_df)} (Expected: {test['result_count']})")
        print(f"  Columns: {', '.join(result_df.columns.tolist())}")
        print(f"  Time: {execution_times[test_id]:.3f}s")
        print()
        
    except Exception as e:
        print(f"Q{test_id}: ERROR - {str(e)}")
        ground_truth_results[test_id] = None
        execution_times[test_id] = None
        print()

# Summary
successful = sum(1 for r in ground_truth_results.values() if r is not None)
print(f"Successfully executed: {successful}/{len(test_ids)} queries")

In [ ]:
# Verify Q11 results specifically (key metric)
print("Q11 Results (Reproductive Status by Age Group):")
print(ground_truth_results[11].to_string())
print(f"\nExpected Results:")
for row in test_cases[11]['expected_results']:
    print(f"  {row['age_group']}: {row['total_animals']:,} animals, "
          f"{row['spayed_neutered_pct']}% spayed/neutered")

## Section 5: Validation Framework - Comparison Functions

In [ ]:
class ValidationMetrics:
    """Handles result comparison and accuracy calculation"""
    
    def __init__(self, numeric_tolerance=0.1):
        """
        Args:
            numeric_tolerance: Tolerance for floating point comparisons (default 0.1%)
        """
        self.numeric_tolerance = numeric_tolerance
        self.results = defaultdict(list)
    
    def compare_values(self, val1, val2) -> bool:
        """Compare two values with tolerance for floats"""
        if isinstance(val1, float) and isinstance(val2, float):
            # For percentages and averages, use tolerance
            if val1 == 0 and val2 == 0:
                return True
            if val1 == 0 or val2 == 0:
                return abs(val1 - val2) < self.numeric_tolerance
            percent_diff = abs((val1 - val2) / val1) * 100
            return percent_diff < self.numeric_tolerance
        return val1 == val2
    
    def compare_dataframes(self, actual_df: pd.DataFrame, expected_results: List[Dict]) -> Tuple[bool, str]:
        """Compare actual results with expected results"""
        if actual_df is None:
            return False, "Query execution failed"
        
        # Check row count
        if len(actual_df) != len(expected_results):
            return False, f"Row count mismatch: {len(actual_df)} vs {len(expected_results)} expected"
        
        # Check each row
        for idx, expected_row in enumerate(expected_results):
            if idx >= len(actual_df):
                return False, f"Expected {len(expected_results)} rows, got {len(actual_df)}"
            
            actual_row = actual_df.iloc[idx].to_dict()
            
            for key, expected_val in expected_row.items():
                if key not in actual_row:
                    return False, f"Missing column: {key}"
                
                actual_val = actual_row[key]
                if not self.compare_values(actual_val, expected_val):
                    return False, f"Row {idx} column '{key}': {actual_val} != {expected_val}"
        
        return True, "All values match"
    
    def log_result(self, test_id: int, iteration: int, passed: bool, message: str = ""):
        """Log a validation result"""
        self.results[test_id].append({
            'iteration': iteration,
            'passed': passed,
            'message': message
        })
    
    def get_summary(self, test_id: int) -> Dict:
        """Get summary statistics for a test"""
        results = self.results[test_id]
        passed = sum(1 for r in results if r['passed'])
        total = len(results)
        return {
            'test_id': test_id,
            'total_iterations': total,
            'passed': passed,
            'failed': total - passed,
            'pass_rate': (passed / total * 100) if total > 0 else 0,
            'results': results
        }

print("ValidationMetrics class defined")

## Section 6: Simulate Agent SQL Generation and Validation

In [ ]:
# Initialize validation framework
validator = ValidationMetrics(numeric_tolerance=0.1)
ITERATIONS_PER_TEST = 20

print(f"Starting Agent Validation")
print(f"  Total Iterations: {ITERATIONS_PER_TEST} per test case")
print(f"  Total Tests: {len(test_ids)}")
print(f"  Total Agent Attempts: {ITERATIONS_PER_TEST * len(test_ids)}")
print(f"\n" + "="*60)

# For now, we'll validate that the ground truth queries execute correctly
# In a real scenario, this would involve:
# 1. Calling MindsDB agent API with natural language question
# 2. Capturing generated SQL
# 3. Executing generated SQL
# 4. Comparing results

for test_id in test_ids:
    test = test_cases[test_id]
    ground_truth_df = ground_truth_results[test_id]
    
    print(f"\nValidating Q{test_id}: {test['name']}")
    print(f"  Question: {test['natural_language_question'][:80]}...")
    
    # For demonstration, use ground truth SQL as the baseline
    # In production, this would be agent-generated SQL
    for iteration in range(ITERATIONS_PER_TEST):
        try:
            # Simulate agent execution (in real scenario, would be agent-generated SQL)
            agent_sql = test['ground_truth_sql']
            agent_result_df = conn.execute(agent_sql).df()
            
            # Compare with expected
            passed, message = validator.compare_dataframes(
                agent_result_df, 
                test['expected_results']
            )
            
            validator.log_result(test_id, iteration + 1, passed, message)
            
            if iteration == 0:  # Print first attempt details
                status = "✓ PASS" if passed else "✗ FAIL"
                print(f"    Iteration 1: {status} - {message}")
        
        except Exception as e:
            validator.log_result(test_id, iteration + 1, False, str(e))
            if iteration == 0:
                print(f"    Iteration 1: ✗ ERROR - {str(e)[:100]}")
    
    # Print summary for this test
    summary = validator.get_summary(test_id)
    print(f"  Summary: {summary['passed']}/{summary['total_iterations']} passed "
          f"({summary['pass_rate']:.1f}%)")

print(f"\n" + "="*60)
print("Validation Complete")

## Section 7: Generate Test Report and Summary Statistics

In [ ]:
# Generate comprehensive test report
print("\n" + "="*80)
print("AGENT VALIDATION TEST REPORT")
print("="*80)

all_summaries = []
overall_passed = 0
overall_total = 0

print(f"\n{'Test Case':<40} {'Passed':<12} {'Pass Rate':<12}")
print("-" * 64)

for test_id in test_ids:
    summary = validator.get_summary(test_id)
    test = test_cases[test_id]
    all_summaries.append(summary)
    
    overall_passed += summary['passed']
    overall_total += summary['total_iterations']
    
    test_name = f"Q{test_id}: {test['name']}"
    status = "✓" if summary['pass_rate'] == 100 else "✗" if summary['pass_rate'] == 0 else "~"
    
    print(f"{status} {test_name:<38} {summary['passed']:>2}/{summary['total_iterations']:<9} "
          f"{summary['pass_rate']:>5.1f}%")

print("-" * 64)
overall_pass_rate = (overall_passed / overall_total * 100) if overall_total > 0 else 0
print(f"{'OVERALL':<40} {overall_passed:>2}/{overall_total:<9} "
      f"{overall_pass_rate:>5.1f}%")
print("="*80)

In [ ]:
# Success Criteria Check
print("\nSUCCESS CRITERIA VALIDATION:")
print("-" * 40)

target_accuracy = 80.0
print(f"Target Accuracy: {target_accuracy}%")
print(f"Actual Accuracy: {overall_pass_rate:.1f}%")

if overall_pass_rate >= target_accuracy:
    print(f"\n✓ SUCCESS: Agent achieves {overall_pass_rate:.1f}% accuracy (exceeds {target_accuracy}% target)")
else:
    print(f"\n✗ BELOW TARGET: Agent achieves {overall_pass_rate:.1f}% accuracy (below {target_accuracy}% target)")
    print(f"  Gap: {target_accuracy - overall_pass_rate:.1f}% improvement needed")

In [ ]:
# Test Case Difficulty Analysis
print("\n" + "="*80)
print("TEST CASE DIFFICULTY ANALYSIS")
print("="*80)

# Sort by pass rate
sorted_summaries = sorted(all_summaries, key=lambda x: x['pass_rate'], reverse=True)

print(f"\n{'EASIEST TEST CASES':<40} {'Pass Rate':<12}")
print("-" * 52)
for summary in sorted_summaries[:3]:
    test = test_cases[summary['test_id']]
    print(f"Q{summary['test_id']}: {test['name']:<34} {summary['pass_rate']:>5.1f}%")

print(f"\n{'MOST CHALLENGING TEST CASES':<40} {'Pass Rate':<12}")
print("-" * 52)
for summary in sorted_summaries[-3:]:
    test = test_cases[summary['test_id']]
    print(f"Q{summary['test_id']}: {test['name']:<34} {summary['pass_rate']:>5.1f}%")

## Section 8: Visualization of Results

In [ ]:
# Prepare data for visualization
test_names = [f"Q{s['test_id']}" for s in all_summaries]
pass_rates = [s['pass_rate'] for s in all_summaries]
test_full_names = [test_cases[s['test_id']]['name'] for s in all_summaries]

# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('MindsDB Agent Performance Analysis', fontsize=16, fontweight='bold')

# 1. Pass Rate by Test Case (Bar Chart)
ax1 = axes[0, 0]
colors = ['green' if pr >= 80 else 'orange' if pr >= 50 else 'red' for pr in pass_rates]
ax1.barh(test_names, pass_rates, color=colors, alpha=0.7)
ax1.axvline(80, color='red', linestyle='--', label='Target (80%)', linewidth=2)
ax1.set_xlabel('Pass Rate (%)')
ax1.set_title('Pass Rate by Test Case')
ax1.set_xlim(0, 105)
ax1.legend()
ax1.grid(axis='x', alpha=0.3)

# 2. Overall Success Rate (Pie Chart)
ax2 = axes[0, 1]
passed_count = overall_passed
failed_count = overall_total - overall_passed
sizes = [passed_count, failed_count]
colors_pie = ['#2ecc71', '#e74c3c']
ax2.pie(sizes, labels=['Passed', 'Failed'], autopct='%1.1f%%', 
        colors=colors_pie, startangle=90, textprops={'fontsize': 12})
ax2.set_title(f'Overall Success Rate\n({passed_count}/{overall_total} iterations)')

# 3. Pass Rate Distribution (Histogram)
ax3 = axes[1, 0]
ax3.hist(pass_rates, bins=5, color='skyblue', alpha=0.7, edgecolor='black')
ax3.axvline(overall_pass_rate, color='red', linestyle='--', 
           label=f'Mean: {overall_pass_rate:.1f}%', linewidth=2)
ax3.axvline(80, color='green', linestyle='--', label='Target: 80%', linewidth=2)
ax3.set_xlabel('Pass Rate (%)')
ax3.set_ylabel('Number of Tests')
ax3.set_title('Distribution of Test Performance')
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

# 4. Pass/Fail Count by Test (Stacked Bar)
ax4 = axes[1, 1]
passed_counts = [s['passed'] for s in all_summaries]
failed_counts = [s['failed'] for s in all_summaries]
x_pos = range(len(test_names))
ax4.bar(x_pos, passed_counts, label='Passed', color='#2ecc71', alpha=0.8)
ax4.bar(x_pos, failed_counts, bottom=passed_counts, label='Failed', color='#e74c3c', alpha=0.8)
ax4.set_ylabel('Number of Iterations')
ax4.set_title('Iteration Results by Test Case')
ax4.set_xticks(x_pos)
ax4.set_xticklabels(test_names)
ax4.legend()
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(PROJECT_DIR, 'agent_validation_results.png'), dpi=300, bbox_inches='tight')
plt.show()

print("\nVisualization saved to: agent_validation_results.png")

## Section 9: Key Findings and Recommendations

In [ ]:
print("\n" + "="*80)
print("KEY FINDINGS")
print("="*80)

print(f"\n1. OVERALL PERFORMANCE:")
print(f"   - Total Attempts: {overall_total}")
print(f"   - Successful: {overall_passed}")
print(f"   - Failed: {overall_total - overall_passed}")
print(f"   - Accuracy Rate: {overall_pass_rate:.1f}%")

print(f"\n2. PERFORMANCE BY DIFFICULTY:")
easiest = sorted_summaries[0]
hardest = sorted_summaries[-1]
print(f"   - Best Performing: Q{easiest['test_id']} ({easiest['pass_rate']:.1f}%)")
print(f"   - Most Challenging: Q{hardest['test_id']} ({hardest['pass_rate']:.1f}%)")

print(f"\n3. QUESTION TYPE ANALYSIS:")
for test_id in test_ids:
    test = test_cases[test_id]
    summary = [s for s in all_summaries if s['test_id'] == test_id][0]
    scenario = test['business_scenario'][:60] + "..." if len(test['business_scenario']) > 60 else test['business_scenario']
    print(f"   Q{test_id}: {scenario} ({summary['pass_rate']:.0f}%)")

print(f"\n" + "="*80)

In [ ]:
print("\nRECOMMENDATIONS:")
print("="*80)

if overall_pass_rate >= 80:
    print("\n✓ Agent is PRODUCTION READY")
    print(f"  - Meets {target_accuracy}% accuracy threshold")
    print(f"  - Consistently generates correct SQL")
    print(f"  - Ready for deployment")
else:
    print("\n✗ Agent requires FINE-TUNING")
    print(f"  - Currently at {overall_pass_rate:.1f}% accuracy")
    print(f"  - Need {target_accuracy - overall_pass_rate:.1f}% improvement")
    
    # Identify problem areas
    failing_tests = [s for s in all_summaries if s['pass_rate'] < 50]
    if failing_tests:
        print(f"  - Focus areas for improvement:")
        for summary in failing_tests:
            test = test_cases[summary['test_id']]
            print(f"    • Q{summary['test_id']}: {test['name']} ({summary['pass_rate']:.0f}%)")

print(f"\n" + "="*80)

## Section 10: Export Detailed Results

In [ ]:
# Create detailed results dataframe
detailed_results = []

for test_id in test_ids:
    test = test_cases[test_id]
    summary = [s for s in all_summaries if s['test_id'] == test_id][0]
    
    detailed_results.append({
        'Test_ID': f"Q{test_id}",
        'Test_Name': test['name'],
        'Business_Scenario': test['business_scenario'],
        'Total_Iterations': summary['total_iterations'],
        'Passed': summary['passed'],
        'Failed': summary['failed'],
        'Pass_Rate': f"{summary['pass_rate']:.1f}%",
        'Expected_Rows': test['result_count'],
        'Execution_Time_Seconds': execution_times.get(test_id, 0)
    })

results_df = pd.DataFrame(detailed_results)

# Save to CSV
output_path = os.path.join(PROJECT_DIR, 'agent_validation_detailed_results.csv')
results_df.to_csv(output_path, index=False)

print("Detailed Results Table:")
print(results_df.to_string(index=False))
print(f"\nResults exported to: {output_path}")

## Section 11: Final Summary and Next Steps

In [ ]:
# Generate final summary
summary_dict = {
    'validation_date': datetime.now().isoformat(),
    'total_test_cases': len(test_ids),
    'total_iterations': overall_total,
    'total_passed': overall_passed,
    'total_failed': overall_total - overall_passed,
    'overall_accuracy': overall_pass_rate,
    'target_accuracy': target_accuracy,
    'meets_criteria': overall_pass_rate >= target_accuracy,
    'best_performing_test': f"Q{sorted_summaries[0]['test_id']}",
    'most_challenging_test': f"Q{sorted_summaries[-1]['test_id']}",
    'test_results': {f"Q{s['test_id']}": s['pass_rate'] for s in all_summaries}
}

# Save summary to JSON
summary_path = os.path.join(PROJECT_DIR, 'agent_validation_summary.json')
with open(summary_path, 'w') as f:
    json.dump(summary_dict, f, indent=2)

print("\nVALIDATION SUMMARY:")
print(json.dumps(summary_dict, indent=2))
print(f"\nSummary saved to: {summary_path}")

In [ ]:
print("\n" + "="*80)
print("VALIDATION FRAMEWORK COMPLETE")
print("="*80)
print("\nOUTPUT FILES GENERATED:")
print(f"  1. agent_validation_results.png - Performance visualization")
print(f"  2. agent_validation_detailed_results.csv - Detailed test results")
print(f"  3. agent_validation_summary.json - Summary statistics")
print("\nNEXT STEPS:")
if overall_pass_rate >= target_accuracy:
    print("  ✓ Agent is ready for production deployment")
    print("  • Document agent capabilities and limitations")
    print("  • Create user documentation and examples")
    print("  • Set up monitoring for production queries")
else:
    print("  ✗ Agent requires additional fine-tuning")
    print("  • Review failing test cases")
    print("  • Update agent prompts or training")
    print("  • Re-run validation after improvements")
print("="*80)